## 1. Import libraries

In [189]:
import numpy as np
import pandas as pd
import time

# machine learning library
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import log_loss
from sklearn.neural_network import MLPClassifier


#import XGBOOST Libraries
import xgboost as xgb
from sklearn.grid_search import GridSearchCV

# plots
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png2x','pdf')
import matplotlib.pyplot as plt

#Configure Panda
pd.options.display.width = 200

## 2. Loading and pre-processing data
### 2.1 Load the files

In [190]:
# This is the input for our model

transactions_feb_untweaked = pd.read_csv('data/transactions_feb.csv')
transactions_mar_untweaked = pd.read_csv('data/transactions_mar_train_v2.csv')
transactions_apr_untweaked = pd.read_csv('data/transactions_apr.csv')
members_feb_untweaked = pd.read_csv('data/members_train1.csv')
members_mar_untweaked = pd.read_csv('data/members_train2.csv')
members_apr_untweaked = pd.read_csv('data/members_total.csv')
user_logs_feb_untweaked = pd.read_csv('data/user_logs_feb.csv')
user_logs_mar_untweaked = pd.read_csv('data/user_logs_mar.csv')
user_logs_apr_untweaked = pd.read_csv('data/user_logs_apr.csv')

print('data loaded!')

data loaded!


del user_logs_feb['pVpp_entries']
del user_logs_mar['pVpp_entries']
del user_logs_apr['pVpp_entries']
del user_logs_feb['pVpp_secs']
del user_logs_mar['pVpp_secs']
del user_logs_apr['pVpp_secs']

del user_logs_feb['entries_m2']
del user_logs_mar['entries_m2']
del user_logs_apr['entries_m2']
del user_logs_feb['secs_m2']
del user_logs_mar['secs_m2']
del user_logs_apr['secs_m2']

In [191]:
print(user_logs_feb_untweaked.shape)
print(user_logs_mar_untweaked.shape)
print(user_logs_apr_untweaked.shape)
print(members_feb_untweaked.shape)
print(members_mar_untweaked.shape)
print(members_apr_untweaked.shape)
print(transactions_feb_untweaked.shape)
print(transactions_mar_untweaked.shape)
print(transactions_apr_untweaked.shape)


(992931, 21)
(970960, 21)
(907471, 21)
(992931, 35)
(970960, 35)
(907471, 35)
(992931, 28)
(970960, 28)
(907471, 28)


##### Testing Zone

In [192]:
user_logs_feb_tweak = user_logs_feb_untweaked.copy()
user_logs_mar_tweak = user_logs_mar_untweaked.copy()
user_logs_apr_tweak = user_logs_apr_untweaked.copy()

members_feb_tweak = members_feb_untweaked.copy()
members_mar_tweak = members_mar_untweaked.copy()
members_apr_tweak = members_apr_untweaked.copy()

transactions_feb_tweak = transactions_feb_untweaked.copy()
transactions_mar_tweak = transactions_mar_untweaked.copy()
transactions_apr_tweak = transactions_apr_untweaked.copy()

For now, ignore this part. We could use it to test the algorithm out by dropping a few features and thereby imrpove the score.

In [193]:
#empirical feature selection improves the result
#user_logs_feb_tweak = user_logs_feb_tweak.drop(['entries_m3','entries_m2','entries_m1','secs_m3','secs_m2','secs_m1'],axis=1)
#user_logs_mar_tweak = user_logs_mar_tweak.drop(['entries_m3','entries_m2','entries_m1','secs_m3','secs_m2','secs_m1'],axis=1)
#user_logs_apr_tweak = user_logs_apr_tweak.drop(['entries_m3','entries_m2','entries_m1','secs_m3','secs_m2','secs_m1'],axis=1)

#members_feb_tweak = members_feb_tweak[['city_1','city_3','city_4','city_22','bd_norm','reg_year_2017','reg_year_2016','reg_year_2015','msno','is_churn']]
#members_mar_tweak = members_mar_tweak[['city_1','city_3','city_4','city_22','bd_norm','reg_year_2017','reg_year_2016','reg_year_2015','msno','is_churn']]
#members_apr_tweak = members_apr_tweak[['city_1','city_3','city_4','city_22','bd_norm','reg_year_2017','reg_year_2016','reg_year_2015','msno','is_churn']]

#transactions_feb_tweak=transactions_feb_tweak[['payment_plan_days_30','transactions','is_cancel','is_auto_renew','msno','is_churn']]
#transactions_mar_tweak=transactions_mar_tweak[['payment_plan_days_30','transactions','is_cancel','is_auto_renew','msno','is_churn']]
#transactions_apr_tweak=transactions_apr_tweak[['payment_plan_days_30','transactions','is_cancel','is_auto_renew','msno','is_churn']]

In [194]:
print(user_logs_feb_tweak.shape)
print(user_logs_mar_tweak.shape)
print(user_logs_apr_tweak.shape)
print(members_feb_tweak.shape)
print(members_mar_tweak.shape)
print(members_apr_tweak.shape)
print(transactions_feb_tweak.shape)
print(transactions_mar_tweak.shape)
print(transactions_apr_tweak.shape)

(992931, 21)
(970960, 21)
(907471, 21)
(992931, 35)
(970960, 35)
(907471, 35)
(992931, 28)
(970960, 28)
(907471, 28)


### 2.2 Merge the different datasets

In this section, the different files containing the features are merged with the train and test data. Here, we need to have a look at the specifics of the problem we actually need to solve. We are given a training data set for february and march with the corresponding response variable "is_churn". We need to use this data and make a prediction for the month of april. To get an estimation of our performance, we are going to train the data on february and test it on march. For the month of april, we are going to train on the entire data, i.e. February and March. The prediction will then be made for april.

In [195]:
feb_data = pd.merge(members_feb_tweak,user_logs_feb_tweak.drop('is_churn',axis=1),on='msno')
feb_data = pd.merge(feb_data,transactions_feb_tweak.drop('is_churn',axis=1),on='msno')
mar_data = pd.merge(members_mar_tweak,user_logs_mar_tweak.drop('is_churn',axis=1),on='msno')
mar_data = pd.merge(mar_data,transactions_mar_tweak.drop('is_churn',axis=1),on='msno')
apr_data = pd.merge(members_apr_tweak,user_logs_apr_tweak.drop('is_churn',axis=1),on='msno')
apr_data = pd.merge(apr_data,transactions_apr_tweak.drop('is_churn',axis=1),on='msno')


##### Checking shapes

In [196]:
print(feb_data.shape)
print(mar_data.shape)
print(apr_data.shape)

(992931, 80)
(970960, 80)
(907471, 80)


### 2.4 Data cleaning

Let's make sure each dataset has the same order of it's columns:

In [197]:
#mar_data = mar_data[feb_data.columns]
apr_data = apr_data[mar_data.columns]


## 3. Algorithms

In [198]:
# Linear Regression
def lin_reg(train,test):
    if ('msno' in train.columns):
        train = train.drop('msno',axis=1)
    if ('msno' in test.columns):
        test = test.drop('msno',axis=1)
    X_train = train.drop('is_churn',axis=1)
    y_train = train['is_churn']
    X_test = test.drop('is_churn',axis=1)
    y_test = test['is_churn']
    
    model_lin_reg = LinearRegression()
    model_lin_reg.fit(X_train, y_train)
    y_pred_l = model_lin_reg.predict(X_test)
    y_pred_l[y_pred_l<0] = 0
    print("Logloss for Linear Regression is: %.6f"%log_loss(y_test,y_pred_l))
    #y_pred_l[y_pred_l>=0.95] = 1
    #y_pred_l[y_pred_l<0.05] = 0
    #print("(1 or 0) Logloss for Linear Regression is: %.6f"%log_loss(y_test,y_pred_l))



In [199]:
lin_reg(feb_data,mar_data)
print('### over-fitting check: ###')
lin_reg(feb_data,feb_data)

Logloss for Linear Regression is: 0.521153
### over-fitting check: ###
Logloss for Linear Regression is: 0.328021


In [200]:
# Run Forest

def ran_for(train,test,importance):
    start = time.time()
    if ('msno' in train.columns):
        train = train.drop('msno',axis=1)
    if ('msno' in test.columns):
        test = test.drop('msno',axis=1)
    X_train = train.drop('is_churn',axis=1)
    y_train = train['is_churn']
    X_test = test.drop('is_churn',axis=1)
    y_test = test['is_churn']
    
    model_Forest = RandomForestRegressor()
    model_Forest.fit(X_train, y_train)
    y_pred_f = model_Forest.predict(X_test)
    print("Logloss for Random Forrest is: %.6f"%log_loss(y_test,y_pred_f))
    
    if (importance == 1):
        importances = model_Forest.feature_importances_
        indices = np.argsort(importances)[::-1]
        columns = np.array(list(X_train))
        return importances
        
        # Print the feature ranking
        print("\nFeature ranking:")
        
        for f in range(x_train.shape[1]):
            print("%d. %s (%f)" % (f + 1, columns[indices[f]], importances[indices[f]]))
    
    print('RF Time = %.0f'%(time.time() - start))


In [201]:
ran_for(feb_data,mar_data,1)
print('### over-fitting check: ###')
ran_for(feb_data,feb_data,0)


Logloss for Random Forrest is: 0.939666
### over-fitting check: ###
Logloss for Random Forrest is: 0.048585
RF Time = 447


In [202]:
# Run XGD Boost
def xgb_boost(train,test,importance,final):
    start = time.time()
    if ('msno' in train.columns):
        train = train.drop('msno',axis=1)
    if ('msno' in test.columns):
        test = test.drop('msno',axis=1)
    X_train = train.drop('is_churn',axis=1)
    y_train = train['is_churn']
    X_test = test.drop('is_churn',axis=1)
    y_test = test['is_churn']
    
    dtrain = xgb.DMatrix(X_train, label = y_train)
    dtest = xgb.DMatrix(X_test, label = y_test)
    param = {
        'max_depth': 3,  # the maximum depth of each tree. Try with max_depth: 2 to 10.
        'eta': 0.3,  # the training step for each iteration. Try with ETA: 0.1, 0.2, 0.3...
        'silent': 1,  # logging mode - quiet
        'objective': 'multi:softprob',  # error evaluation for multiclass training
        'num_class': 3}  # the number of classes that exist in this datset
    num_round = 20  # the number of training iterations. Try with num_round around few hundred!
    #----------------
    bst = xgb.train(param, dtrain, num_round)
    y_pred_xgb = bst.predict(dtest)
    best_preds = np.asarray([np.argmax(line) for line in y_pred_xgb])
    y_pred_xgb = y_pred_xgb[:,1] #Column 2 out of 3
    if (final == 1):
        return y_pred_xgb
    print("(probs) Logloss for XGD Boost is: %.6f"%log_loss(y_test,y_pred_xgb))
    
    if (importance == 1):
        xgb.plot_importance(bst,max_num_features=10)
        plt.show()
    
    #y_pred_xgb[y_pred_xgb>=0.5] = 1
    #y_pred_xgb[y_pred_xgb<0.5] = 0
    #print("(1 or 0) Logloss for XGD Boost is: %.6f"%log_loss(y_test,y_pred_xgb))

    print('XGB Time = %.0f'%(time.time() - start))
    
    return y_pred_xgb

xgb_boost(feb_data,mar_data,1,0)
print('### over-fitting check: ###')
xgb_boost(feb_data,feb_data,0,0)

In [203]:
# Neural Network
def neural(train,test,cv):
    start = time.time()
    if ('msno' in train.columns):
        train = train.drop('msno',axis=1)
    if ('msno' in test.columns):
        test = test.drop('msno',axis=1)
    X_train = train.drop('is_churn',axis=1)
    y_train = train['is_churn']
    X_test = test.drop('is_churn',axis=1)
    y_test = test['is_churn']

    model_n = MLPClassifier()
    model_n.fit(X_train, y_train)
    y_pred_n = model_n.predict(X_test)
    
    print("Logloss for Neural Network is: %.6f"%log_loss(y_test,y_pred_n)) 
    
    if (cv == 1):
        cv = ShuffleSplit(n_splits=3, test_size=0.33, random_state=42)
        scores = cross_val_score(model_n, data_input, data_output, cv=cv, scoring='log_loss')
        #scores = - scores
        print(scores)
        print("CV log loss: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    
    print('Neural Time = %.0f'%(time.time() - start))

In [204]:
neural(feb_data,mar_data,0)
print('### over-fitting check: ###')
neural(feb_data,feb_data,0)

Logloss for Neural Network is: 8.863182
Neural Time = 23
### over-fitting check: ###
Logloss for Neural Network is: 2.196548
Neural Time = 41


In [205]:
# AdaBoost
def ada_b(train,test):
    start = time.time()
    if ('msno' in train.columns):
        train = train.drop('msno',axis=1)
    if ('msno' in test.columns):
        test = test.drop('msno',axis=1)
    X_train = train.drop('is_churn',axis=1)
    y_train = train['is_churn']
    X_test = test.drop('is_churn',axis=1)
    y_test = test['is_churn']
    
    model_abr = AdaBoostRegressor()
    model_abr.fit(X_train, y_train)
    print('Model fitted!')
    y_pred_abr = model_abr.predict(X_test)
    print("Logloss for AdaBoost is: %.6f"%log_loss(y_test,y_pred_abr))
    print('Time =',time.time() - start)

In [206]:
ada_b(feb_data,mar_data)
print('### over-fitting check: ###')
ada_b(feb_data,feb_data)

Model fitted!
Logloss for AdaBoost is: 0.241840
Time = 140.39595460891724
### over-fitting check: ###
Model fitted!
Logloss for AdaBoost is: 0.183722
Time = 119.57487607002258


As user_logs data contains daterelated features that are related to the date where is_churn is taken from, and we have is_churn at two different dates, we can append mar_data and feb_data and thus will have a trainset of double the size.

In [207]:
total_data = mar_data.append(feb_data)

Below we will create the final submission file:

In [208]:
#Prepare submission file
y_pred_xgb = xgb_boost(total_data,apr_data,0,1)
my_submission = pd.DataFrame({'msno': apr_data.msno, 'is_churn': y_pred_xgb})
print(my_submission.head())
cols = my_submission.columns.tolist()
cols = cols[-1:] + cols[:-1]
my_submission = my_submission[cols]
print(my_submission.head())
print(my_submission.count())

my_submission.to_csv('submission.csv', index=False)
print('Done! :-)')

   is_churn                                          msno
0  0.026196  4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=
1  0.016431  aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=
2  0.072834  rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=
3  0.023867  WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=
4  0.068619  aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=
                                           msno  is_churn
0  4n+fXlyJvfQnTeKXTWT507Ll4JVYGrOC8LHCfwBmPE4=  0.026196
1  aNmbC1GvFUxQyQUidCVmfbQ0YeCuwkPzEdQ0RwWyeZM=  0.016431
2  rFC9eSG/tMuzpre6cwcMLZHEYM89xY02qcz7HL4//jc=  0.072834
3  WZ59dLyrQcE7ft06MZ5dj40BnlYQY7PHgg/54+HaCSE=  0.023867
4  aky/Iv8hMp1/V/yQHLtaVuEmmAxkB5GuasQZePJ7NU4=  0.068619
msno        907471
is_churn    907471
dtype: int64
Done! :-)
